In [1]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
# from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

## Part 1
Provide plots of the joint-angles versus time for the following gait parameters.

<img src="MAE207 HW3 Gait1.png">

<img src="MAE207 HW3 Gait2.png">

<img src="MAE 207 HW3 Gait3.png">

The below code was used to generate these plots.  The get_gait function takes xy coordinates and returns joint angles

In [2]:
l1 = 0.09;                  
l2 = 0.16;              
w = 0.07;   

In [3]:
# solve jacobian of constraint equation
(thetaL_sym, 
 thetaR_sym, 
 link1_sym, 
 link2_sym, 
 width_sym) = sp.symbols("""thetaL_sym 
                            thetaR_sym 
                            link1_sym 
                            link2_sym 
                            width_sym""", real = True)

In [4]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [5]:
#  get forward kinematics of 5-bar leg

x_r = width_sym/2 + link1_sym*sp.cos(thetaR_sym)
x_l = -width_sym/2 + link1_sym*sp.cos(thetaL_sym)

y_r = link1_sym*sp.sin(thetaR_sym)
y_l = link1_sym*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_r - y_l, x_r - x_l)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, -width_sym/2, 0)@T(-(thetaL_sym - theta3_sym), link1_sym, 0)@sp.Matrix([L/2, sp.sqrt(link2_sym**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()
FK

Matrix([
[                                                                  link1_sym*((sin(thetaL_sym) - sin(thetaR_sym))*sqrt(2*link1_sym**2*cos(thetaL_sym - thetaR_sym) - 2*link1_sym**2 + 2*link1_sym*width_sym*cos(thetaL_sym) - 2*link1_sym*width_sym*cos(thetaR_sym) + 4*link2_sym**2 - width_sym**2)/sqrt(-2*link1_sym**2*cos(thetaL_sym - thetaR_sym) + 2*link1_sym**2 - 2*link1_sym*width_sym*cos(thetaL_sym) + 2*link1_sym*width_sym*cos(thetaR_sym) + width_sym**2) + cos(thetaL_sym) + cos(thetaR_sym))/2],
[link1_sym*(-sin(thetaL_sym) + sin(thetaR_sym))/2 + link1_sym*sin(thetaL_sym) + (-link1_sym*cos(thetaL_sym) + link1_sym*cos(thetaR_sym) + width_sym)*sqrt(2*link1_sym**2*cos(thetaL_sym - thetaR_sym) - 2*link1_sym**2 + 2*link1_sym*width_sym*cos(thetaL_sym) - 2*link1_sym*width_sym*cos(thetaR_sym) + 4*link2_sym**2 - width_sym**2)/(2*sqrt(-2*link1_sym**2*cos(thetaL_sym - thetaR_sym) + 2*link1_sym**2 - 2*link1_sym*width_sym*cos(thetaL_sym) + 2*link1_sym*width_sym*cos(thetaR_sym) + width_sym**2))

In [6]:
# function for getting Inverse Kinematics of 5-link leg
def IK_5_link(x, y, l1 = l1, l2 = l2, w = w):
    
    def leg_wide(var):
        return np.linalg.norm([var[1] - np.pi, var[0]])
    
    def x_constraint_equation(var):
        # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                               {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))

x = np.array([-.10, -.9])
y = np.array([.10, .11])

res = IK_5_link(x,y)
print(res)

(     fun: 0.3296908309475615
     jac: array([ 0.30331449, -0.95289052])
 message: 'More equality constraints than independent variables'
    nfev: 4
     nit: 1
    njev: 1
  status: 2
 success: False
       x: array([0.1       , 2.82743339]), 1.1907007771827451)


In [114]:
# function that returns joint angles required for gait made up of a line and parabola
def get_gait(coords, d=0.5, T=0.25, N=100):
    
    xy_coords = []
    
    x_coords = coords[:, 0]
    y_coords = coords[:,1]
    
    lin_eqn = np.poly1d(np.polyfit(x_coords[:2], y_coords[:2], deg=1))
    
    parab_eqn = np.poly1d(np.polyfit(x_coords, y_coords, deg=2))
    
    lin_ticks = d * N
    parab_ticks = N - lin_ticks
    
    # get line coordinates
    
    line_dist = np.sqrt((coords[0,0] - coords[1,0])**2 + (coords[0,1] - coords[1,1])**2)
    dist_per_tick = line_dist / lin_ticks

    line_angle = np.arcsin(np.abs(coords[0,1] - coords[1,1]) / line_dist)

    direction = 1
    if x_coords[1] < x_coords[0]:
        direction = -1
    
    start = [coords[0,0], coords[0,1]]
    xy_coords.append(start)
    
    for i in range(int(lin_ticks)):
        x_prev = xy_coords[i][0]
        y_prev = xy_coords[i][1]
        x_new = x_prev + direction * dist_per_tick * np.cos(line_angle)
        y_new = lin_eqn(x_new)
        
        xy_coords.append([x_new, y_new])
      
    # get parabola coordinates
    
    x_change = np.abs((coords[0,0] - coords[1,0])) / parab_ticks
    x_start = coords[1,0] - direction * x_change
    y_start = parab_eqn(x_start)
    start = [x_start, y_start]
    ind_start = len(xy_coords)

    xy_coords.append(start)
    
    for i in range(int(parab_ticks - 1)):
        x_prev = xy_coords[i+ind_start][0]
        y_prev = xy_coords[i+ind_start][1]
        
        x_new = x_prev - direction*x_change
        y_new = parab_eqn(x_new)
        
        xy_coords.append([x_new, y_new])
    
    
    joint_angles = [] 
    # Convert xy points to joint angles 
    for i in range(len(xy_coords)):
        x = xy_coords[i][0]
        y = xy_coords[i][1]
        res = IK_5_link(x,y)
        
        joint_angles.append([res[0].x[0],res[0].x[1]])
    
    
    xy_coords_array = np.array(xy_coords)
    joint_angles_array = np.array(joint_angles)
       
    # plot joint angles over time
    plt.figure(1)
    plt.plot(np.linspace(0,T,N+1), joint_angles_array[:,0], 'go', label='Theta R')
    plt.plot(np.linspace(0,T,N+1), joint_angles_array[:,1], 'bo', label='Theta L')
    plt.xlabel('Time (s)')
    plt.ylabel('Joint Angle (rad)')
    plt.title('Joint Angles over Time')
    plt.legend()
    
    return joint_angles_array


In [104]:
# Failed attempt at making the points uniformly spread by time
d= 0.5
N = 100
coords = np.array([[-0.11,0.12],[0.11,0.12],[0,0.05]])
x_coords = coords[:, 0]
y_coords = coords[:,1]
parab_eqn = np.poly1d(np.polyfit(x_coords, y_coords, deg=2))
lin_ticks = d * N
parab_ticks = N - lin_ticks

parab_dist = np.poly1d(np.sqrt(1 + np.square(np.poly1d.deriv(parab_eqn)))).integ()

total_dist = np.abs(parab_dist(x_coords[0]) - parab_dist(x_coords[1]))
print(total_dist)
dist_per_tick = total_dist / parab_ticks
print(dist_per_tick)



0.22
0.0044


In [105]:
# Get joint angles for Gait 1
coordinates = np.array([[-0.11,0.12],[0.11,0.12],[0,0.05]])
joint_angles = get_gait(coordinates, d=0.5, T=0.25, N=100)

In [115]:
# Get joint angles for Gait 2
coordinates = np.array([[-0.08,0.10],[0.11,0.14],[0.05,0.05]])
joint_angles = get_gait(coordinates, d=0.75, T=0.5, N=100)

In [107]:
# Get joint angles for Gait 3
coordinates = np.array([[-0.08,0.10],[0.08,0.10],[0,0.04]])
joint_angles = get_gait(coordinates, d=0.5, T=1.5, N=200)

## Part 2
1. Collect and turn in a video of one of the gaits from the list above.
    - emailed in

2. Record the motor positions for all of the gaits above. Provide a plot of the commanded
motor position and actual motor position vs time for each gait.

# Gait 1

<img src="MAE207 HW3 Gait 1 ThetaR.png">

<img src="MAE207 HW3 Gait1 ThetaL.png">

# Gait 2

<img src="MAE207 HW3 Gait2 ThetaR.png">

<img src="MAE207 HW3 Gait2 ThetaL.png">

# Gait 3

<img src="MAE207 HW3 Gait3 ThetaR.png">

<img src="MAE207 HW3 Gait3 ThetaL.png">

Below is the code used to control, read from the motors in this section, and generate the above plots

In [108]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
# from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
import odrive.utils
import odrive.enums

In [111]:
odrv0 = odrive.find_any()
if odrv0 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv0.serial_number))
    
    m0 = odrv0.axis0.motor.is_calibrated
    m1 = odrv0.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')

Connected!
Odrive serial 53215608451637
Motor 0 calibrated: False
Motor 1 calibrated: False


In [112]:
odrv0.axis0.controller.config.vel_limit = 200000
odrv0.axis1.controller.config.vel_limit = 200000

In [113]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Motor 0 calibration result: {} \r\n'.format(odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(odrv0.axis1.motor.is_calibrated))

	 Motor 0 calibration result: True 
 	 Motor 1 calibration result: True


In [284]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

odrv0.axis0.controller.set_pos_setpoint(1.2343835830688477,0,0)
odrv0.axis1.controller.set_pos_setpoint(-3207.765625,0,0)

In [50]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

In [116]:
# adjust joint angles

adjusted_joint_angles = np.pi - joint_angles

In [119]:
motor_cpr = (odrv0.axis0.encoder.config.cpr, 
             odrv0.axis1.encoder.config.cpr)

def convert_joints(angles, cpr=motor_cpr, zero_pos = (2774.234375-4096, -1452.25)):
    encoder_vals = (angles * cpr[0] / (2 * np.pi)) + zero_pos
    return encoder_vals

In [117]:
motor_cpr = (odrv0.axis0.encoder.config.cpr, 
             odrv0.axis1.encoder.config.cpr)

def get_joints(odrv, cpr = motor_cpr, zero_pos = (0,0)):
    m0 = 2*np.pi*(odrv.axis0.encoder.pos_estimate - zero_pos[0])/motor_cpr[0]
    m1 = 2*np.pi*(odrv.axis1.encoder.pos_estimate - zero_pos[1])/motor_cpr[1]
    
    return (m0, m1)

joints = get_joints(odrv0)

print('encoder0: ', odrv0.axis0.encoder.pos_estimate)
print('encoder1: ', odrv0.axis1.encoder.pos_estimate)
print('angles in rad: ',joints)
print('converted to encoder: ', convert_joints(np.array(list(joints))))

print('test: ', convert_joints(np.array([0,0])))

encoder0:  2774.234375
encoder1:  -1452.25
angles in rad:  (2.1278111161709647, -1.1138617996034612)
converted to encoder:  [ 2479.984375   -8804.25097656]
test:  [ -294.25       -7352.00097656]


In [120]:
# get encoder values
encoder_vals = convert_joints(adjusted_joint_angles)

In [125]:
import math
T = 0.5
N = 100
time_per_tick = T / N

odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

actual_motor_pos0 = []
actual_motor_pos1 = []

for j in range(5):
    for i in range(encoder_vals.shape[0]):
        start = time.time()
        thetaR = encoder_vals[i,0]
        thetaL = encoder_vals[i,1]
        if math.isnan(thetaR):
            print("nan")
        else:

            odrv0.axis0.controller.set_pos_setpoint(thetaR,0,0)
            odrv0.axis1.controller.set_pos_setpoint(thetaL,0,0)
            
        M0 = odrv0.axis0.encoder.pos_estimate
        M1 = odrv0.axis1.encoder.pos_estimate
        
        actual_motor_pos0.append(M0)
        actual_motor_pos1.append(M1)

        end = time.time()

        time.sleep(time_per_tick - (start - end))
    

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [123]:
plt.plot(np.linspace(0,T,N),encoder_vals[1:,0], 'ro', label='Desired')
plt.plot(np.linspace(0,T,N),actual_motor_pos0[1:], 'go', label='Actual')
plt.title("Motor 0 (Theta R) Desired and Actual Position")
plt.xlabel('Time (s)')
plt.ylabel('Motor position (encoder ticks)')
plt.legend()
plt.plot()

[]

In [124]:
plt.plot(np.linspace(0,T,N),encoder_vals[1:,1], 'ro', label='Desired')
plt.plot(np.linspace(0,T,N),actual_motor_pos1[1:], 'go', label='Actual')
plt.title("Motor 1 (Theta L) Desired and Actual Position")
plt.xlabel('Time (s)')
plt.ylabel('Motor position (encoder ticks)')
plt.legend()
plt.plot()

[]

In [740]:
odrive.utils.dump_errors(odrv0)

axis0
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
axis1
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
